# Idea:
Our solution: LDA + BERT based embeddings of noun phrases and verbs :
- Each noun phrase and verb in the texts is  transformed to embedding vector using Universal Sentence Encoder (transformer based on BERT)
- Embedding vectors from (a) are clustered (HDBSCAN + UNET)
- Words/phrases with embedding vectors closest to the centers of resulting clusters form key word/phrase
- Each text in the training sample is converted to collection of key-phrases by replacing its noun phrases and verbs with keyword/phrases and deleting other words
- LDA is performed on the transformed texts


**Reference:**<br>
- Daniel Cer, Yinfei Yang, Sheng-yi Kong, Nan Hua, Nicole Limtiaco, Rhomni St. John, Noah Constant, Mario Guajardo-Céspedes, Steve Yuan, Chris Tar, Yun-Hsuan Sung, Brian Strope, Ray Kurzweil. **Universal Sentence Encoder.** *arXiv:1803.11175, 2018.*
- McInnes, L, Healy, J, **UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction**, *ArXiv e-prints 1802.03426, 2018*

# Load data and python libraries

In [1]:
# data processing libraries
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# display wider columns in pandas data frames where necessary
pd.set_option('max_colwidth',150)

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import tensorflow_hub as hub
#Load the Universal Sentence Encoder's TF Hub module
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
model = hub.load(module_url)
print ("module %s loaded" % module_url)

import umap
import hdbscan

TensorFlow version: 2.2.0
module https://tfhub.dev/google/universal-sentence-encoder-large/5 loaded


In [2]:
df_train = pd.read_csv("./data/train_grouped.tsv", sep="\t")
print("df_train.shape:", df_train.shape)
print("df_train.shape:",df_train.columns)

df_train.shape: (33982, 16)
df_train.shape: Index(['date', 'author', 'title', 'url', 'section', 'publication',
       'first_10_sents', 'list_of_first_10_sents', 'list_of_verb_lemmas',
       'noun_phrases', 'list_of_nouns', 'list_of_lemmas', 'ID',
       'group_level_1', 'group_level_2', 'group_level_3'],
      dtype='object')


# Getting text clusters through sentence embedding comparison

In [3]:
def get_embeddings(input):
    return model(input)

In [4]:
def get_word_embeddings(df_data, column = "word", N_batches=1):
    #split data into N batches
    N = N_batches

    part = int(len(df_data)/N)
    print(N, "batches with", part + 1, column + "s each")

    #get embeddings for each N words
    index = 0
    batch_num = 0
    list_dfs = []

    while index < len(df_data): 
        df_tmp = df_data.iloc[index : index + part].copy()
        df_tmp = df_tmp.reset_index(drop=True)
        print ("Batch number:", batch_num + 1, "out of ", N)

        df_batch_embeddings = pd.DataFrame(get_embeddings(list(df_tmp[column])).numpy())

        num_embeddings = df_batch_embeddings.shape[1]
        columns = ["emb_" + str(i) for i in range(512)]
        df_tmp[columns] = df_batch_embeddings

        list_dfs.append(df_tmp)
        batch_num = batch_num + 1
        index = index + part

    #concatinate batches into single dataset
    df_emb = pd.concat(list_dfs)

    return df_emb

In [5]:
df_train['noun_phrases'] = df_train['noun_phrases'].str[2:-2]
df_train['noun_phrases'] = df_train['noun_phrases'].str.lower().str.split("', '")
df_train['noun_phrases'].head()

0    [rise, big emerging economy, china, india, steady march, globalisation, surge, number, people, business, tourism, result, demand, visa, unpreceden...
1    [pfizer, commitment, corporate social responsibility csr, drugs giant talk, responsibility, society, world, access, product, work, ngos, global he...
2    [week, federal reserve, interest rate, time, year, world, central bank, rate, recent year, long spell, course, chart, outcome, americas rate rise,...
3    [cruise line, wave, year, nearly, holiday, sea, result, december 18th carnival, worlds largest operator, global market, fullyear earning, demand, ...
4    [investors, calendar year, buoyant mood, unexpected event, consensus, respect, view, investor, market price, column, potential surprise, definitio...
Name: noun_phrases, dtype: object

In [6]:
all_NPs = list(df_train['noun_phrases'])
all_NPs = [np for l in all_NPs for np in l if len(np)>0]
all_NPs[:5], len(all_NPs)

(['rise', 'big emerging economy', 'china', 'india', 'steady march'], 1417049)

In [7]:
df_train['list_of_verb_lemmas'].iloc[0]

'[emerging, led, wanting, travel, granted, Upgrade, travel, apply, submit, streamline, scrap]'

In [8]:
df_train['list_of_verb_lemmas'] = df_train['list_of_verb_lemmas'].str[2:-2]
df_train['list_of_verb_lemmas'] = df_train['list_of_verb_lemmas'].str.lower().str.split(", ")
df_train['list_of_verb_lemmas'].head()

0                                                               [merging, led, wanting, travel, granted, upgrade, travel, apply, submit, streamline, scra]
1    [rided, embracing, insists, gain, strengthen, improve, deterred, seeking, intends, shift, domiciled, rejoiced, saved, paid, outraged, promised, im...
2    [aised, ended, celebrate, tried, lift, forced, reverse, cut, help, understand, upgrade, strike, wish, save, spend, try, escape, slashing, encourag...
3      [race, booked, improve, announced, control, demand, peaking, piling, based, got, moving, upgrade, increase, announced, establish, aimed, based, ad]
4    [tart, caught, proved, reflected, like, suggest, judged, betting, expect, upgrade, weakens, having, pushed, tighten, buy, priced, doubt, tighten, ...
Name: list_of_verb_lemmas, dtype: object

In [9]:
all_Vs = list(df_train['list_of_verb_lemmas'])
all_Vs = [v for l in all_Vs for v in l if len(v)>0]
all_Vs[:5], len(all_Vs)

(['merging', 'led', 'wanting', 'travel', 'granted'], 675330)

In [10]:
all_words =  list(set(all_NPs + all_Vs))
len(set(all_words))

419327

In [11]:
df_words = pd.DataFrame({'word': all_words})
df_words.head()

,word
0,illegal trade
1,universal free health care
2,deadly consequence
3,users activity
4,soonthe company


In [12]:
%%time
#creating word2vec matrix
df_w2v = get_word_embeddings(df_words, column = "word", N_batches=100)
df_w2v.head()

100 batches with 4194 words each
Batch number: 1 out of  100
Batch number: 2 out of  100
Batch number: 3 out of  100
Batch number: 4 out of  100
Batch number: 5 out of  100
Batch number: 6 out of  100
Batch number: 7 out of  100
Batch number: 8 out of  100
Batch number: 9 out of  100
Batch number: 10 out of  100
Batch number: 11 out of  100
Batch number: 12 out of  100
Batch number: 13 out of  100
Batch number: 14 out of  100
Batch number: 15 out of  100
Batch number: 16 out of  100
Batch number: 17 out of  100
Batch number: 18 out of  100
Batch number: 19 out of  100
Batch number: 20 out of  100
Batch number: 21 out of  100
Batch number: 22 out of  100
Batch number: 23 out of  100
Batch number: 24 out of  100
Batch number: 25 out of  100
Batch number: 26 out of  100
Batch number: 27 out of  100
Batch number: 28 out of  100
Batch number: 29 out of  100
Batch number: 30 out of  100
Batch number: 31 out of  100
Batch number: 32 out of  100
Batch number: 33 out of  100
Batch number: 34 ou

,word,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_502,emb_503,emb_504,emb_505,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511
0,illegal trade,0.088174,-0.024202,0.032902,0.034414,-0.043001,0.047338,-0.000035,0.001882,-0.032775,...,-0.025584,0.000864,0.052648,-0.022472,0.045635,0.049618,0.009245,0.020719,0.010762,0.031821
1,universal free health care,-0.013799,0.077828,0.028418,0.013109,0.051120,0.016538,0.017926,-0.027106,0.028668,...,-0.038434,0.050330,0.164079,0.023516,-0.037081,0.026098,-0.019342,0.034655,0.068513,0.082887
2,deadly consequence,0.053178,-0.005348,-0.008809,-0.018735,-0.015988,-0.001234,-0.017554,0.020279,-0.061362,...,-0.042913,-0.037350,-0.006825,0.072847,-0.053072,0.063832,0.025820,0.027221,-0.054562,0.014381
3,users activity,-0.026125,0.016817,0.005812,0.067683,0.071939,0.070370,-0.053353,0.005770,0.060505,...,0.015617,-0.031431,-0.006912,-0.024569,0.085909,-0.026211,0.053759,-0.108147,-0.009249,-0.034792
4,soonthe company,-0.009154,-0.008781,0.026742,-0.005074,-0.028769,0.041692,0.090339,-0.052231,-0.007637,...,0.010862,0.013613,0.014772,-0.023711,0.005862,0.031451,-0.012725,-0.102446,0.021036,0.100148


In [13]:
df_w2v.iloc[::150001]

,word,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_502,emb_503,emb_504,emb_505,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511
0,illegal trade,0.088174,-0.024202,0.032902,0.034414,-0.043001,0.047338,-0.000035,0.001882,-0.032775,...,-0.025584,0.000864,0.052648,-0.022472,0.045635,0.049618,0.009245,0.020719,0.010762,0.031821
3246,american film studio,0.002946,0.080105,0.009157,0.008486,0.007905,0.099968,-0.016649,-0.058844,-0.048068,...,0.009001,0.017807,0.068661,0.015315,-0.032000,-0.041465,-0.036790,-0.008068,0.060914,0.032913
2299,analyzed hospital discharge datum,-0.026752,0.055819,0.042151,0.044713,-0.014527,-0.101770,0.006108,-0.059279,-0.059515,...,0.002860,0.021987,-0.047937,0.030521,0.013224,0.000185,0.034310,-0.061723,-0.013282,0.015742


# Dimensionality reduction 
UMAP https://github.com/lmcinnes/umap

In [14]:
columns = ["emb_" + str(i) for i in range(512)]
embeddings = df_w2v[columns].values
embeddings.shape

(419327, 512)

In [15]:
%%time
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)
umap_embeddings.shape

CPU times: user 57min 46s, sys: 1min 21s, total: 59min 8s
Wall time: 6min 8s


(419327, 5)

# Clustering

In [16]:
cluster = hdbscan.HDBSCAN(min_cluster_size=15,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)

In [17]:
#cluster labels
labels = cluster.labels_
labels.shape

(419327,)

In [18]:
#number of clusters (key-words/phrases)
print("Number of clusters:",labels.max() + 1)

Number of clusters: 4319


# Prepare data for LDA

##### get cluster label as most frequent word/phrase of the cluster

In [19]:
df_tmp = df_w2v[['word']].copy()
df_tmp['cluster_number'] = labels
df_tmp.head()

,word,cluster_number
0,illegal trade,-1
1,universal free health care,3430
2,deadly consequence,-1
3,users activity,3461
4,soonthe company,4038


In [20]:
df_all_words = pd.DataFrame({'word': list(set(all_NPs + all_Vs))})
df_all_words = df_all_words.merge(df_tmp, on='word', how='inner')
print(df_all_words.shape)
df_all_words.head()

(419327, 2)


,word,cluster_number
0,illegal trade,-1
1,universal free health care,3430
2,deadly consequence,-1
3,users activity,3461
4,soonthe company,4038


In [21]:
#label cluster with noise words as "noise"
def get_label(n,w):
    if n == -1:
        return "noise"
    else:
        return w

df_all_words['cluster_label'] = df_all_words.apply(lambda x: get_label(x.cluster_number, x.word), axis=1) 
df_all_words.iloc[::222222]

,word,cluster_number,cluster_label
0,illegal trade,-1,noise
222222,teslas laserwindex,53,teslas laserwindex


In [22]:
df_all_words['word_frequency'] = df_all_words.groupby(['cluster_number', 
                                                       'cluster_label'])['word'].transform("count")
df_all_words['word_max_frequency'] = df_all_words.groupby(['cluster_number'])['word_frequency'].transform("max")
df_all_words.iloc[::222222]

,word,cluster_number,cluster_label,word_frequency,word_max_frequency
0,illegal trade,-1,noise,163822,163822
222222,teslas laserwindex,53,teslas laserwindex,1,1


In [23]:
pd.DataFrame(df_all_words.describe(percentiles=[0.01,0.1,0.20,0.3,0.4,0.5,0.75,0.95,0.99])).T

,count,mean,std,min,1%,10%,20%,30%,40%,50%,75%,95%,99%,max
cluster_number,419327.0,1356.103366,1488.978787,-1.0,-1.0,-1.0,-1.0,-1.0,57.0,801.0,2628.0,4146.0,4318.0,4318.0
word_frequency,419327.0,64002.325605,79928.760269,1.0,1.0,1.0,1.0,1.0,1.0,1.0,163822.0,163822.0,163822.0,163822.0
word_max_frequency,419327.0,64002.325605,79928.760269,1.0,1.0,1.0,1.0,1.0,1.0,1.0,163822.0,163822.0,163822.0,163822.0


In [24]:
df_all_words['word_max_frequency'] = df_all_words.groupby(['cluster_number'])['word_frequency'].transform("max")

df_tmp_noise = pd.DataFrame(df_all_words[df_all_words['cluster_label'] == "noise"])
df_tmp_other = df_all_words[df_all_words['cluster_label'] != "noise"]
print(len(df_tmp_noise),len(df_tmp_other))
df_tmp_noise.head()

163823 255504


,word,cluster_number,cluster_label,word_frequency,word_max_frequency
0,illegal trade,-1,noise,163822,163822
2,deadly consequence,-1,noise,163822,163822
5,outer solar system moon,-1,noise,163822,163822
7,fertilizer,-1,noise,163822,163822
12,central draw,-1,noise,163822,163822


In [25]:
df_tmp = df_tmp_other[df_tmp_other['word_max_frequency'] == df_tmp_other['word_frequency']]
df_tmp = df_tmp.groupby('cluster_number')['cluster_label'].last().reset_index()
print(df_tmp.shape)
df_tmp.iloc[::1000]

(4319, 2)


,cluster_number,cluster_label
0,0,related hashtag
1000,1000,good game
2000,2000,poopthe upgrade
3000,3000,companys ideological echochamber
4000,4000,bill simmons new site


In [26]:
df_word_clusters = df_all_words[['word', 'cluster_number']]
df_word_clusters = df_word_clusters.merge(df_tmp, on='cluster_number', how='left')

df_word_clusters['cluster_label'] = df_word_clusters['cluster_label'].fillna("noise")

print(df_word_clusters.shape)
df_word_clusters.iloc[::222222].T

(419327, 3)


,0,222222
word,illegal trade,teslas laserwindex
cluster_number,-1,53
cluster_label,noise,tesla roadster payload


In [27]:
df_word_clusters = df_word_clusters.drop_duplicates('word')
df_word_clusters.shape

(419327, 3)

In [28]:
s = df_word_clusters.groupby('cluster_label')['word'].count()
s.describe()

count      4320.000000
mean         97.066435
std        2497.764297
min          15.000000
25%          23.000000
50%          36.000000
75%          65.000000
max      163822.000000
Name: word, dtype: float64

In [29]:
s.tail()

cluster_label
zerowaste airport       80
ziggy stardust year     18
zika hot zone          107
ziploc bag              19
zombies                 35
Name: word, dtype: int64

In [30]:
df_word_clusters[df_word_clusters['cluster_label'] == "zombie internet"]

,word,cluster_number,cluster_label


In [31]:
#create word -> cluster_name series
del df_word_clusters['cluster_number']

word_cluster_label = df_word_clusters.set_index("word")
print(len(word_cluster_label))
word_cluster_label.head()

419327


,cluster_label
word,
illegal trade,noise
universal free health care,envision healthcare
deadly consequence,noise
users activity,users basic information
soonthe company,ukrainebased company


In [32]:
word_cluster_label.to_csv('./transition_files/word_cluster_label.csv')